In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

sys.path.append("../")
from utils.io import read_parameters
from src.plot_trajectories import plot_trajectories_per_file
from src.plot_trajectories import plot_trajectories_from_origin_per_file
from src.plot_trajectories import plot_trajectories_from_origin_per_condition
from src.plot_trajectories import get_color_from_speed

cmap = matplotlib.colormaps["seismic"]

In [ ]:
# read parameters and key file

parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)       

key_file_path = parameters["key_file"]
key_file = pd.read_csv(key_file_path)

print("key file columns:", key_file.columns)
print("key file conditions:", key_file["condition"].unique())

In [ ]:
# create output folder

output_folder = parameters["output_folder"]
print("Output folder:", output_folder)

data_folder = output_folder + "/tracking_data/"

subfolder = "trajectory_plots"
if not os.path.exists(output_folder + "/" + subfolder):
    os.mkdir(output_folder + "/" + subfolder)

#tracking_data = pd.read_csv(data_folder + "tracking_data.csv") 


In [ ]:
# plot paramters

condition = 'mosaic_siCTRL_siAlk1'
color = 'red'
treatment = 'siAlk1'

number_of_tracks_per_condition = 1000


In [ ]:
#key_select = key_file[key_file["condition"] == ]
#key_select = key_select[key_select["color"] == color]
key_select = key_file[key_file["treatment"] == treatment] 

number_of_tracks_per_file = int(number_of_tracks_per_condition/len(key_select.index))
print("sample %s tracks per file" % number_of_tracks_per_file)

track_counter = 0
center_x = 0
center_y = 0

fig, ax = plt.subplots(figsize=(9,9))

for index, row in key_select.iterrows():

    tracking_file = "tracking_data_%s_%s_%s.csv" % (treatment, row["color"], row["experimentID"])
                
    data = pd.read_csv(data_folder+ tracking_file, low_memory=False)

    observation_period_df = data[data["FRAME"] <= parameters["end_phase_1"]]

    trackID_list = np.array(observation_period_df["TRACK_ID"].unique())
    print("Available tracks: %s" % len(trackID_list))
    trackID_list = np.random.choice(trackID_list, number_of_tracks_per_file)

    print("sampled %s tracks for file %s" % (len(trackID_list),tracking_file))
    track_counter += len(trackID_list)

    center_per_file_x = 0.0
    center_per_file_y = 0.0
    total_dist_per_file = 0.0
                
    for track_id in trackID_list:
        single_track_df = observation_period_df[observation_period_df["TRACK_ID"] == track_id]

        end_x = np.array(single_track_df["ORIGIN_X"])[-1]
        end_y = np.array(single_track_df["ORIGIN_Y"])[-1]

        total_dist_per_file += np.sqrt(end_x**2 + end_y**2)

        center_x += end_x
        center_y += end_y

        center_per_file_x += end_x
        center_per_file_y += end_y

        rel_vel = get_color_from_speed(-end_x)

                    # ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = "#ff7f00")
                    # ax.plot([end_x],[end_y], color = "black", marker = "o")

                    # color by track id             
                    #ax.plot(single_track_df["ORIGIN_X"], single_track_df["ORIGIN_Y"])
                    #ax.plot([end_x], [end_y], marker="o")

                    # color by velocity   
        ax.plot(single_track_df["ORIGIN_X"],single_track_df["ORIGIN_Y"], color = cmap(rel_vel))
        ax.plot([end_x],[end_y], color = "black", marker = "o", alpha=0.5) 


ax.set_xlim(-110,110)
ax.set_ylim(-110,110)
ax.axhline(0, color = "red", linestyle = "--")
ax.axvline(0, color = "red", linestyle = "--")
ax.set_title(treatment)
ax.set_xlabel("$\Delta x$ in $\mu m$")
ax.set_ylabel("$\Delta y$ in $\mu m$")
ax.set_aspect(1)
plt.tight_layout()

In [ ]:
#single_condition_data = tracking_data[tracking_data["condition"] == condition]
#single_color_data = single_condition_data[single_condition_data["color"] == color]
#single_color_data = single_color_data[data["FRAME"] <= parameters["observation_period"][1]]
#ingle_color_data = single_color_data[data["FRAME"] >= parameters["observation_period"][0]]

In [ ]:
# plot_trajectories_from_origin_per_condition(parameters, key_file, subfolder = "tracking_data", number_of_tracks_per_condition = parameters["number_of_cells_per_condition"])